In [1]:
import cv2
import time
import numpy as np
from os import listdir
from os.path import isfile, join

In [2]:
# detect face in photo
def face_detect(photo):
    # face detection in photo
    model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    face= model.detectMultiScale(photo)

    if len(face) == 0:
        print("No face detected")
        return
    else:
        x1 = face[0][0]
        y1 = face[0][1]
        x2 = face[0][2]
        y2 = face[0][3]
        photo = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
        crop_photo = photo[y1:y1+y2,x1:x1+x2]     
        crop_photo = cv2.resize(crop_photo,(200,200))
        
        return crop_photo 

In [3]:
import smtplib
# And imghdr to find the types of our images
import imghdr

# Here are the email package modules we'll need
from email.message import EmailMessage
import PersonalDetails as p
import importlib
importlib.reload(p)

# Create the container email message.
def send_email():
    
    msg1 = EmailMessage()
    msg1['From'] = p.sender_email()
    msg1['To'] = p.Receiver_email()
    msg1['Subject'] = 'Face Recognition'
    msg1['Body']  = 'This is face of Manisha'
    #msg1.attach(text)
    with open('image.png', 'rb') as fp:
        img_data = fp.read()
        img_type = imghdr.what(fp.name)
        img_name = fp.name
    msg1.add_attachment(img_data,maintype='image',subtype=img_type)

    # Send the email via our own SMTP server.
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as s:
        s.login(p.sender_email(),p.Password())
        s.send_message(msg1)
    return True

In [4]:
# send whatsapp mess
def whatsapp():    
    import pywhatkit as kit
    import PersonalDetails as p
    import importlib
    importlib.reload(p)
    text="Successfully complete the task 6"
    kit.sendwhatmsg_instantly(p.Whatsapp_no(),text,2)
    return True

In [5]:
# collect data set
count =1
cap = cv2.VideoCapture(0)
while True:
    ret, photo = cap.read()
    #cap.release()
    crop_photo = face_detect(photo)
    if crop_photo is not None:
        crop_photo = cv2.putText(crop_photo,str(count),(0, 180),cv2.FONT_HERSHEY_SIMPLEX ,1, (0, 0, 255), 2)

        myphoto = "./faces/manisha{}.png".format(count)
        cv2.imwrite(myphoto,crop_photo)

        cv2.imshow("Photo",crop_photo)
        count=count+1
        if cv2.waitKey(1)== 13 or count == 151:
            cap.release()
            break
cv2.destroyAllWindows()
print("Successful Dataset created")

Successful Dataset created


In [6]:
dataset = './faces/'
images = [img for img in listdir(dataset) if isfile(join(dataset, img))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(images):
    image_path = dataset + images[i]
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(image, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

#model  = cv2.face_LBPHFaceRecognizer.create()
model = cv2.face.LBPHFaceRecognizer_create()

# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully....")

Model trained sucessefully....


In [7]:
#if photo not found
import os,subprocess
def launch_ec2():
    os.system('aws ec2 run-instances --image-id ami-0ad704c126371a549 --count 1 --instance-type t2.micro --key-name Mynewkey --security-group-ids sg-0d5dec3dfdecf3be6')
    return 

In [8]:
def ebs_volume():
    az = subprocess.getoutput('aws ec2 describe-instances --filters Name=instance-state-name,Values=running --query "Reservations[].Instances[].Placement.AvailabilityZone" --output text')
    os.system('aws ec2 create-volume --availability-zone  {} --size 5'.format(az))
    return 

In [9]:
def attach_volume():
    instance_id = subprocess.getoutput('aws ec2 describe-instances --filters Name=instance-state-name,Values=running --query "Reservations[].Instances[].InstanceId" --output text')
    volume_id = subprocess.getoutput('aws ec2 describe-volumes --filters Name=size,Values=5 --query "Volumes[*].VolumeId" --output text')
    time.sleep(20)
    os.system('aws ec2 attach-volume --device /dev/sdh --instance-id {} --volume-id {}'.format(instance_id, volume_id))
    return 

In [10]:
def notfound(photo):
    cv2.putText(photo, "Not Match", (0, 180), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Not Match', photo )
    cv2.waitKey(1)            
    time.sleep(5)
    cv2.destroyAllWindows()
    import os
    import subprocess
    if launch_ec2() == None:
        print("Successfully launch ec2 instance")
        time.sleep(30)
    if ebs_volume() == None:
        print("Successfully launch Ebs volume")
        time.sleep(90)
    if attach_volume() == None:
        print("Successfully attached volume")
        time.sleep(10) 

In [11]:
def detect(photo):
    crop_photo =photo
    crop_photo = cv2.resize(photo,(200,200))
    crop_photo = cv2.cvtColor(crop_photo, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Photo",crop_photo)
    cv2.waitKey()
    cv2.destroyAllWindows()
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(crop_photo)
    if len(faces) !=0:
        for (x,y,w,h) in faces:
            cv2.rectangle(photo,(x,y),(x+w,y+h),(0,255,0),3)
            ids,conf = model.predict(crop_photo[y:y+h,x:x+w])
            print("face detected successfully")
        if conf < 500 :
            conf = int( 100 * (1 - (conf/400) ))
            print("confidence = ",conf,"%")
        if conf > 70:
            cv2.putText(crop_photo, "Manisha", (0, 180), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', crop_photo )
            cv2.waitKey(1)            
            time.sleep(5)
            cv2.destroyAllWindows()
            cv2.imwrite('image.png',crop_photo)
            if send_email() == True:
                time.sleep(5)
                print("Successfully mailed")
            if whatsapp() == True:
                time.sleep(15)
                print("Successfully send whatsapp mess")
        else:
            return crop_photo
            
    else:
        return crop_photo
        

In [12]:
cap = cv2.VideoCapture(0)
ret, photo = cap.read()
cap.release()
detect(photo)

face detected successfully
confidence =  77 %
Successfully mailed
Successfully send whatsapp mess


In [13]:
ph = cv2.imread('swastik1.jpg')
cv2.imshow("hii",ph)
cv2.waitKey()
cv2.destroyAllWindows()
img = detect(ph)
if img.any() != None:
    notfound(img)


face detected successfully
confidence =  65 %
Successfully launch ec2 instance
Successfully launch Ebs volume
Successfully attached volume


Thank you:)